In [ ]:
from googleapiclient.discovery import build
import pandas as pd


DEVELOPER_KEY = "chiaveAPI"
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, 
                YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY
               )

# ID del canale di Joe Rogan
channel_id = 'UCzQUP1qoWDoEbmsQxvdjxgQ'
playlist_id = 'PLk1Sqn_f33KuWf3tW9BBe_4TP7x8l0m3T'

def get_all_videos_from_playlist(youtube, playlist_id):
    video_ids = []
    next_page_token = None

    while True:
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

    return video_ids

def extract_guest_name(title):
    if " - " in title:
        return title.split(" - ", 1)[1].strip()
    elif " with " in title:
        return title.split(" with ", 1)[1].strip()
    return None


def get_video_details(youtube, video_ids):
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,statistics,topicDetails',
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            title = video['snippet']['title']
            if 'Joe Rogan Experience #' not in title:
                continue
            guest_name = extract_guest_name(title)
            
            if guest_name is None:
                continue
                
            video_info = {
                'video_id': video['id'],
                'title': title,
                'guest': guest_name,
                'published_at': video['snippet']['publishedAt'],
                'views': video['statistics'].get('viewCount'),
                'likes': video['statistics'].get('likeCount'),
                'comments': video['statistics'].get('commentCount'),
                'description': video['snippet']['description'],
            }
            all_video_info.append(video_info)

    return all_video_info


video_ids = get_all_videos_from_playlist(youtube, playlist_id)
video_details = get_video_details(youtube, video_ids)

df = pd.DataFrame(video_details)
print(df)

df.to_csv("joe_rogan_episodes.csv", index=False)


         video_id                                              title  \
0     C81bFx8CSA8            Joe Rogan Experience #2334 - Kash Patel   
1     EDBFFgs6Ifs  Joe Rogan Experience #2333 - Protect Our Parks 15   
2     mYvGKBCM3Ps           Joe Rogan Experience #2332 - Oz Pearlman   
3     r9Ldl70x5Fg         Joe Rogan Experience #2331 - Jesse Michels   
4     qe64ayAbDjM                  Joe Rogan Experience #2330 - Bono   
...           ...                                                ...   
2068  t4UJ8rMW_u0          Joe Rogan Experience #301 - Doug Stanhope   
2069  p0zNBUF-nR4       Joe Rogan Experience #304 - Andrew Dice Clay   
2070  3F7y0U6L6cI             Joe Rogan Experience #308 - Steve Volk   
2071  ZpHh_TU2kcI    Joe Rogan Experience #310 - Neil Degrasse Tyson   
2072  WPt2Ec1QJlk  Joe Rogan Experience #312 - Steve Rinella, Bry...   

                            guest          published_at    views  likes  \
0                      Kash Patel  2025-06-06T17:00:16Z  263